In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [22]:
# create the model 
import modeldefine
input_size = 5
hidden_size = 10 
output_size = 1
num_layers = 5

model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)
print(model)

Model(
  (layers): ModuleList(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): ReLU()
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): ReLU()
    (10): Linear(in_features=10, out_features=1, bias=True)
    (11): ReLU()
  )
)


In [23]:
params = list(model.parameters())

# 打印每个参数的形状
for param in params:
    print(param.shape)
    print(param)

torch.Size([20, 10])
Parameter containing:
tensor([[ 0.0059, -0.3591,  0.2124, -0.0511,  0.6059,  0.0000, -0.0000,  0.0000,
         -0.0000,  0.0000],
        [-0.1608,  0.5579, -0.6366, -0.3898, -0.0051, -0.0000,  0.0000, -0.0000,
         -0.0000, -0.0000],
        [-0.1308,  0.5288,  0.1113,  0.2322, -0.1095, -0.0000,  0.0000,  0.0000,
          0.0000, -0.0000],
        [-0.4892, -0.2016, -0.0771,  0.2091, -0.2249, -0.0000, -0.0000, -0.0000,
          0.0000, -0.0000],
        [-0.1482,  0.4760,  0.3872, -0.5996,  0.3546, -0.0000,  0.0000,  0.0000,
         -0.0000,  0.0000],
        [ 0.2342,  0.2154,  0.1482, -0.3483, -0.2084,  0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 0.1843,  0.1121,  0.1667, -0.2389, -0.3233,  0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000],
        [-0.1486, -0.0247,  0.4259, -0.1226, -0.2484, -0.0000, -0.0000,  0.0000,
         -0.0000, -0.0000],
        [ 0.9412,  0.0572, -0.2189,  0.0379,  0.1230,  0.0000,  0.0000, -0.0000,
    

In [25]:
data = torch.randn(1, ).to(device)
model(data)

RuntimeError: self and mat2 must have the same dtype, but got Float and Double